### С delphinet какие-то проблемы, там не получается нормально запустить _тот же самый контракт_. Пока не понял в чём проблема
- во-первых при пересоздании контракта возникает ошибка proto.007-PsDELPH1.michelson_v1.invalid_arity, которой нет в эдонете
- во-вторых при попытках сделать ставки ошибка JSONDecodeError: Expecting value: line 1 column 1 (char 0)
    * может быть просто pytezos не дружит с делфинет? Или как-то нужно в явном виде указывать что надо работать с этой сетью?
- в третьих при закрытии контракта ошибка: proto.007-PsDELPH1.michelson_v1.script_rejected

In [1]:
from pytezos import pytezos
from keys_manager import KeysManager
from contract_manager import ContractManager
from settings import settings

contract_storage = settings['DEFAULT_INITIAL_CONTRACT_STORAGE']
contract_storage.update({'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx'})

settings.update(dict(
    CONTRACT_ADDRESS = 'KT1StGt3uerh9wkiwoaNed3dmQhyEmKKnP4r',
    SHELL_URL = 'https://delphinet.smartpy.io/',
    DEFAULT_INITIAL_CONTRACT_STORAGE = contract_storage
))

### Загрузка ключей
Ключи были заранее подготовлены с помощью https://faucet.tzalpha.net/

Если ключи не были активированы, можно выполнить команду
```python keys.activate_keys() ```

In [2]:
keys = KeysManager(settings)

Successfully loaded 5 pytezos keys:
- tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE
- tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM
- tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos
- tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ
- tz1ZAzDvkZCT2LAyPN8Kdxw3kes7xfWerZhZ


### Загружается имеющийся в сети контракт, который я задеплоил сегодня утром:

In [3]:
pt = keys.pytezos_instances['tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM']
cm = ContractManager(keys.pytezos_instances['tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM'], settings)

In [4]:
storage = cm.contract.storage()
storage

{'betsAgainstLedger': 83876,
 'betsAgainstSum': 0,
 'betsForLedger': 83877,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
 'targetRate': 0,
 'targetTime': 0}

### Это уже закрытый контракт на который я отправил несколько ставок и скоторого уже были выведены средства

## Создание нового контракта:
#### Ставка на то что сегодня после 10 часов по Киеву, в момент закрытия контракта (который может вызвать любой участник сети) курс Tezos на бирже Coinbase превысит 3.5 доллара USD:

- currencyPair: это ценовая пара, запрашиваемая в оракле, в нашем случае XTZ-USD
- oracleAddress: это адрес Harbinger Price Oracle в сети edonet (не оффициальный, оффициальный есть только в delphinet и mainnet) с нормализованными ценами биржи Coinbase
- targetRate: это цена Tezos в долларах, при достижении которой в targetTime
- targetTime: это unix time, значению 1614542400 соответствует 28 февраля, 22 часа по Киеву

In [5]:
'''new_storage = cm.create_new_storage(
    currencyPair='XTZ-USD',
    oracleAddress='KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
    targetRate=4_000_000,
    targetTime=1614542400
)'''

new_storage = cm.contract.storage()
new_storage.update(dict(targetRate=4_000_000, targetTime=1614542400))

# new_contract = cm.pytezos.origination(script=cm.contract.script(initial_storage=new_storage))
new_contract = cm.pytezos.origination(script=cm.contract.script())

In [6]:
# cm.contract.script(initial_storage=new_storage)

In [7]:
# import pdb; pdb.run('new_contract.autofill()')

In [8]:
new_storage

{'betsAgainstLedger': 83876,
 'betsAgainstSum': 0,
 'betsForLedger': 83877,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
 'targetRate': 4000000,
 'targetTime': 1614542400}

In [9]:
new_contract.autofill()#.sign().inject(_async=False)

MichelsonError: ({'expected_arity': 2,
  'id': 'proto.007-PsDELPH1.michelson_v1.invalid_arity',
  'kind': 'permanent',
  'location': 0,
  'primitive_name': 'Pair',
  'wrong_arity': 4},)

In [7]:
ncm = cm.deploy_new_contract(
    currencyPair='XTZ-USD',
    oracleAddress='KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
    targetRate=4_000_000,
    targetTime=1614542400
)

Deploying new contract with storage:
{'betsAgainstLedger': 0,
 'betsAgainstSum': 0,
 'betsForLedger': 0,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
 'targetRate': 4000000,
 'targetTime': 1614542400}


MichelsonError: ({'expected_arity': 2,
  'id': 'proto.007-PsDELPH1.michelson_v1.invalid_arity',
  'kind': 'permanent',
  'location': 0,
  'primitive_name': 'Pair',
  'wrong_arity': 4},)

In [5]:
ncm = cm

### Ставки на контракты:
- Ставка на то, что курс не достигнет 3.5 USD составляет 0.414 тезоса:

In [6]:
ncm.contract.betAgainst().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)

Wait 11 seconds until block BMPP4A2Zy4EsDzytFZexhLFGP8jstwhJuD6LLPGATdQTpTM9sgX is finalized


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

- Ставка на то, что курс превысит 3.5 USD составляет 1 тезос:

In [7]:
# TODO: здесь надо поменять кошелёк, но я не успеваю:
ncm.contract.betFor().with_amount(1_000_000).as_transaction().autofill().sign().inject(_async=False)

Wait 0 seconds until block BLiNd4xzdyVfgq2GhnTMqAazjzCr8CKr1VrHXW8XyVi26UBobZg is finalized


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Текущее состояние контракта:

In [8]:
ncm.contract.storage()

{'betsAgainstLedger': 83876,
 'betsAgainstSum': 414000,
 'betsForLedger': 83877,
 'betsForSum': 1000000,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
 'targetRate': 0,
 'targetTime': 0}

### Попытка закрыть контракт:

In [9]:
ncm.contract.close().as_transaction().autofill().sign().inject(_async=False)

MichelsonError: ({'id': 'proto.007-PsDELPH1.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 95,
  'with': {'prim': 'Unit'}},)

### Ошибка - потому что ещё не достигнуто время. В тот момент когда у оракула будет ценовая пара с timestamp > чем заявленный в смарт контракте - можно будет закрыть контракт и затем запустить вывод средств:

In [ ]:
ncm.withdraw().as_transaction().autofill().sign().inject(_async=False)

Я проверял - механизм работает. Однозначно нужно написать тесты и закрыть некоторые пробелы. Также нужно сделать удобный интерфейс для работы с контрактами, площадку на которой можно создавать такие контракты и делать ставки на них.